# ARCA Beverage Demo: Model Deployment Strategies

This notebook demonstrates different deployment options for ML models in Snowflake after training with Many Model Training (MMT).

**Deployment Options Covered:**
1. **Model Registry Verification** - Confirm models and versions exist
2. **Model Aliases** - Version management with production/staging aliases
3. **Batch Inference** - Scheduled and on-demand predictions using SQL
4. **Blue-Green Deployment** - Zero-downtime model updates with traffic shifting

**Prerequisites:**
- Completed notebook `04_many_model_training.ipynb`
- Models registered in Model Registry

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
import pandas as pd
import os

session = get_active_session()
session.sql("USE DATABASE ARCA_BEVERAGE_DEMO").collect()
session.sql("USE SCHEMA ML_DATA").collect()
session.sql("USE WAREHOUSE ARCA_DEMO_WH").collect()

registry = Registry(
    session=session,
    database_name="ARCA_BEVERAGE_DEMO",
    schema_name="MODEL_REGISTRY"
)
print("✅ Connected to Snowflake and Model Registry")

## 1. Verify Model Registry Setup

Before working with deployment strategies, let's verify that the models from notebook 04 exist in the registry.

Set 'PRODUCTION' alias for V1 models trained in notebook 04.

In [ ]:
expected_models = [f"WEEKLY_SALES_FORECAST_SEGMENT_{i}" for i in range(1, 7)]

print("🔍 Verifying Model Registry Setup\n")

models_df = session.sql("SHOW MODELS IN SCHEMA ARCA_BEVERAGE_DEMO.MODEL_REGISTRY").collect()
existing_models = [row['name'] for row in models_df]

missing = [m for m in expected_models if m not in existing_models]
if missing:
    print(f"❌ Missing models: {missing}")
    print("   Please run notebook 04_many_model_training.ipynb first")
else:
    print(f"✅ All {len(expected_models)} expected models found in registry")
    for name in expected_models:
        print(f"   • {name}")

### 1a. Explore Segment 1 Model Versions and Tags

Let's examine the model versions and their associated metadata for the Segment 1 forecast model.

In [ ]:
model_name = "WEEKLY_SALES_FORECAST_SEGMENT_1"
model = registry.get_model(model_name)

print(f"📦 Model: {model_name}\n")
print("=" * 70)

versions = model.show_versions()
print(f"\n🔢 Versions ({len(versions)} total):\n")
for _, v in versions.iterrows():
    print(f"   Version: {v['name']}")
    print(f"   Created: {v['created_on']}")
    print(f"   Comment: {v.get('comment', 'N/A')}")
    print()

print("=" * 70)
print("\n🏷️  Aliases:\n")
for alias_name in ['DEFAULT', 'FIRST', 'LAST']:
    try:
        alias_version = model.version(alias_name)
        print(f"   {alias_name:12} → {alias_version.version_name}")
    except:
        print(f"   {alias_name:12} → (not set)")

## 2. Model Aliases for Version Management

Model aliases provide a way to manage model versions without changing downstream code. This enables pre-production validation by running candidate models alongside production before promotion.

**Our Alias Strategy:**
- **CANDIDATE** - Newest version being validated before promotion
- **PRODUCTION** - Current live model serving predictions (also set as DEFAULT)
- **BACKUP** - Previous production version for quick rollback

```
Model Lifecycle with Aliases:
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  CANDIDATE  │───▶│ PRODUCTION  │───▶│   BACKUP    │
│  (newest)   │    │ (2nd newest)│    │  (oldest)   │
└─────────────┘    └─────────────┘    └─────────────┘
   Validation         Serving           Rollback
   & Testing          Traffic           Safety
```

In [ ]:
print("🏷️  Setting Aliases Based on Version Recency\n")

for segment_id in range(1, 7):
    model_name = f"WEEKLY_SALES_FORECAST_SEGMENT_{segment_id}"
    model = registry.get_model(model_name)
    
    versions = model.show_versions().sort_values('created_on', ascending=False)
    version_names = versions['name'].tolist()
    
    for v_name in version_names:
        for alias in ['CANDIDATE', 'PRODUCTION', 'BACKUP']:
            try:
                model.version(v_name).unset_alias(alias)
            except:
                pass
    
    alias_mapping = {
        'CANDIDATE': version_names[0],   # newest
        'PRODUCTION': version_names[1],  # 2nd newest
        'BACKUP': version_names[2]       # 3rd newest (oldest)
    }
    
    for alias, version in alias_mapping.items():
        model.version(version).set_alias(alias)
    
    model.default = alias_mapping['PRODUCTION']
    
    print(f"✅ {model_name}")
    for alias, version in alias_mapping.items():
        print(f"      {alias:12} → {version}")
    print(f"      {'DEFAULT':12} → {alias_mapping['PRODUCTION']}")

In [ ]:
print("🔍 Verify Alias Configuration\n")

for segment_id in range(1, 7):
    model_name = f"WEEKLY_SALES_FORECAST_SEGMENT_{segment_id}"
    model = registry.get_model(model_name)
    versions = model.show_versions()
    
    print(f"📦 {model_name}")
    print(f"   {'Version':<25} {'Aliases'}")
    print("   " + "-" * 50)
    for _, v in versions.iterrows():
        aliases = v.get('aliases', '[]')
        print(f"   {v['name']:<25} {aliases}")
    print()

### 2a. Parallel Model Comparison: Candidate vs Production

Compare CANDIDATE and PRODUCTION models on the same live data to validate before promotion.

In [ ]:
print("📊 Parallel Comparison: CANDIDATE vs PRODUCTION for Segment 1\n")

model_name = "WEEKLY_SALES_FORECAST_SEGMENT_1"
model = registry.get_model(model_name)

test_data = session.table("ARCA_BEVERAGE_DEMO.ML_DATA.TRAINING_DATA").filter(
    "SEGMENT = 'SEGMENT_1'"
)

feature_cols = [
    'CUSTOMER_TOTAL_UNITS_4W', 'WEEKS_WITH_PURCHASE', 'VOLUME_QUARTILE',
    'WEEK_OF_YEAR', 'MONTH', 'QUARTER', 'TRANSACTION_COUNT',
    'UNIQUE_PRODUCTS_PURCHASED', 'AVG_UNITS_PER_TRANSACTION'
]

eval_df = test_data.select(['WEEKLY_SALES_UNITS'] + feature_cols)
features_df = test_data.select(feature_cols)

print(f"Evaluation records: {eval_df.count():,}\n")

In [ ]:
from snowflake.snowpark.functions import col, abs as sf_abs, pow as sf_pow, sqrt, mean, lit

candidate_mv = model.version('CANDIDATE')
production_mv = model.version('PRODUCTION')

print(f"CANDIDATE version:  {candidate_mv.version_name}")
print(f"PRODUCTION version: {production_mv.version_name}\n")

candidate_preds = candidate_mv.run(features_df, function_name="predict")
production_preds = production_mv.run(features_df, function_name="predict")

candidate_pdf = candidate_preds.to_pandas()
production_pdf = production_preds.to_pandas()
actuals_pdf = eval_df.to_pandas()

candidate_pdf['ACTUAL'] = actuals_pdf['WEEKLY_SALES_UNITS']
candidate_pdf['MODEL'] = 'CANDIDATE'
candidate_pdf = candidate_pdf.rename(columns={'output_feature_0': 'PREDICTION'})

production_pdf['ACTUAL'] = actuals_pdf['WEEKLY_SALES_UNITS']
production_pdf['MODEL'] = 'PRODUCTION'
production_pdf = production_pdf.rename(columns={'output_feature_0': 'PREDICTION'})

In [ ]:
import numpy as np

def calculate_metrics(actual, predicted):
    mae = np.mean(np.abs(actual - predicted))
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mape = np.mean(np.abs((actual - predicted) / np.where(actual == 0, 1, actual))) * 100
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

candidate_metrics = calculate_metrics(candidate_pdf['ACTUAL'], candidate_pdf['PREDICTION'])
production_metrics = calculate_metrics(production_pdf['ACTUAL'], production_pdf['PREDICTION'])

print("📈 Performance Comparison\n")
print(f"{'Metric':<10} {'PRODUCTION':>15} {'CANDIDATE':>15} {'Difference':>15}")
print("-" * 60)

for metric in ['MAE', 'RMSE', 'MAPE']:
    prod_val = production_metrics[metric]
    candidate_val = candidate_metrics[metric]
    diff = candidate_val - prod_val
    indicator = "⬇️" if diff < 0 else "⬆️" if diff > 0 else "➡️"
    unit = "%" if metric == "MAPE" else ""
    print(f"{metric:<10} {prod_val:>14.2f}{unit} {candidate_val:>14.2f}{unit} {diff:>+14.2f} {indicator}")

print("\n⬇️ = CANDIDATE is better (lower error)")
print("⬆️ = PRODUCTION is better (lower error)")

In [ ]:
print("📋 Sample Predictions Comparison\n")

comparison_df = pd.DataFrame({
    'ACTUAL': candidate_pdf['ACTUAL'].head(10),
    'PRODUCTION': production_pdf['PREDICTION'].head(10),
    'CANDIDATE': candidate_pdf['PREDICTION'].head(10)
})
comparison_df['PROD_ERROR'] = abs(comparison_df['ACTUAL'] - comparison_df['PRODUCTION'])
comparison_df['CANDIDATE_ERROR'] = abs(comparison_df['ACTUAL'] - comparison_df['CANDIDATE'])

print(comparison_df.to_string(index=False))

### 2b. Promote Candidate to Production (Alias Approach)

After validating CANDIDATE performs well, promote it to PRODUCTION and demote current PRODUCTION to BACKUP.

In [ ]:
print("🔄 Promoting CANDIDATE → PRODUCTION → BACKUP\n")

model_name = "WEEKLY_SALES_FORECAST_SEGMENT_1"
model = registry.get_model(model_name)

candidate_version = model.version('CANDIDATE').version_name
production_version = model.version('PRODUCTION').version_name
backup_version = model.version('BACKUP').version_name

print(f"Before promotion:")
print(f"   CANDIDATE:  {candidate_version}")
print(f"   PRODUCTION: {production_version}")
print(f"   BACKUP:     {backup_version}\n")

model.version(candidate_version).unset_alias('CANDIDATE')
model.version(production_version).unset_alias('PRODUCTION')
model.version(backup_version).unset_alias('BACKUP')

model.version(candidate_version).set_alias('PRODUCTION')
model.version(production_version).set_alias('BACKUP')

model.default = candidate_version

print(f"After promotion:")
print(f"   PRODUCTION: {candidate_version} (was CANDIDATE)")
print(f"   BACKUP:     {production_version} (was PRODUCTION)")
print(f"   DEFAULT:    {candidate_version}")
print(f"\n⚠️  Note: CANDIDATE is now unassigned until next model version is trained")

print("🔍 Verify Promotion\n")

model = registry.get_model(model_name)
versions = model.show_versions()

print(f"{'Version':<25} {'Aliases'}")
print("-" * 50)
for _, v in versions.iterrows():
    aliases = v.get('aliases', '[]')
    print(f"{v['name']:<25} {aliases}")

### 2c. Segment Promotion Strategy

When managing segmented models, consider two approaches:

**Synchronized Promotion (Recommended)**
- All segments promoted together as a unit
- Best when segments are trained together with shared hyperparameters (as in MMT)
- Simpler operational model with easier rollback and auditing
- One promotion decision applies to all segments

**Independent Promotion**
- Each segment promoted on its own schedule
- Better when segments have different data volumes, drift rates, or tuning needs
- More operational complexity but finer-grained control
- Risk of configuration drift between segments

Since our MMT approach trains all segments together with the same hyperparameters, synchronized promotion is the natural fit.

In [ ]:
def promote_all_segments():
    """Atomically promote CANDIDATE → PRODUCTION → BACKUP for all segments."""
    print("🔄 Synchronized Promotion: All Segments\n")
    
    for segment_id in range(1, 7):
        model_name = f"WEEKLY_SALES_FORECAST_SEGMENT_{segment_id}"
        model = registry.get_model(model_name)
        
        candidate_ver = model.version('CANDIDATE').version_name
        prod_ver = model.version('PRODUCTION').version_name
        backup_ver = model.version('BACKUP').version_name
        
        model.version(candidate_ver).unset_alias('CANDIDATE')
        model.version(prod_ver).unset_alias('PRODUCTION')
        model.version(backup_ver).unset_alias('BACKUP')
        
        model.version(candidate_ver).set_alias('PRODUCTION')
        model.version(prod_ver).set_alias('BACKUP')
        model.default = candidate_ver
        
        print(f"✅ {model_name}: {candidate_ver} → PRODUCTION")
    
    print("\n✅ All segments promoted successfully")

print("📦 Function defined: promote_all_segments()")
print("   Promotes CANDIDATE → PRODUCTION and PRODUCTION → BACKUP for all 6 segments")

### 2d. Rolling Batch Inference (Canary for Warehouse)

For batch/warehouse inference, we can implement canary-style validation by routing a percentage of inputs to the CANDIDATE model while the rest use PRODUCTION. This allows validating the new model on real data before full promotion.

```
Rolling Batch Strategy:
┌─────────────────────────────────────────────────┐
│              INPUT DATA (100%)                  │
└─────────────────────┬───────────────────────────┘
                      │
         ┌────────────┴────────────┐
         │ HASH(customer_id) % 5   │
         └────────────┬────────────┘
                      │
        ┌─────────────┴─────────────┐
        ▼                           ▼
   ┌─────────┐                 ┌─────────┐
   │   20%   │                 │   80%   │
   │CANDIDATE│                 │PRODUCTION│
   └─────────┘                 └─────────┘
```

This approach ensures:
- **Deterministic routing**: Same customer always goes to same model (consistent experience)
- **Representative sample**: Hash-based split distributes evenly across customer segments
- **Easy comparison**: Can compare predictions/outcomes between the two groups

In [ ]:
def rolling_batch_inference(
    model_name: str,
    input_table: str,
    output_table: str,
    candidate_pct: int = 20,
    id_column: str = "CUSTOMER_ID"
):
    """
    Run batch inference with a percentage of records routed to CANDIDATE model.
    Uses deterministic hashing so the same customer always gets the same model.
    
    Args:
        model_name: Model name in registry
        input_table: Source table with features
        output_table: Destination table for predictions
        candidate_pct: Percentage of records for CANDIDATE (default 20%)
        id_column: Column to hash for deterministic routing
    """
    model = registry.get_model(model_name)
    production_mv = model.version('PRODUCTION')
    candidate_mv = model.version('CANDIDATE')
    
    input_df = session.table(input_table)
    
    hash_mod = 100 // candidate_pct
    
    candidate_df = input_df.filter(f"MOD(HASH({id_column}), {hash_mod}) = 0")
    production_df = input_df.filter(f"MOD(HASH({id_column}), {hash_mod}) != 0")
    
    print(f"📊 Rolling Batch Inference: {candidate_pct}% CANDIDATE / {100-candidate_pct}% PRODUCTION")
    print(f"   CANDIDATE records: {candidate_df.count():,}")
    print(f"   PRODUCTION records: {production_df.count():,}\n")
    
    candidate_preds = candidate_mv.run(candidate_df, function_name="predict")
    candidate_preds = candidate_preds.with_column("MODEL_VERSION", lit(candidate_mv.version_name))
    
    production_preds = production_mv.run(production_df, function_name="predict")
    production_preds = production_preds.with_column("MODEL_VERSION", lit(production_mv.version_name))
    
    all_preds = candidate_preds.union_all(production_preds)
    all_preds.write.mode("overwrite").save_as_table(output_table)
    
    print(f"✅ Predictions written to {output_table}")
    print(f"   Total records: {all_preds.count():,}")
    
    return {
        "candidate_version": candidate_mv.version_name,
        "production_version": production_mv.version_name,
        "candidate_count": candidate_df.count(),
        "production_count": production_df.count()
    }

print("📦 Function defined: rolling_batch_inference(model_name, input_table, output_table, candidate_pct)")
print("   Routes percentage of batch to CANDIDATE model for validation")

## 3. Gateways as Model Endpoints

Snowflake Gateways provide stable HTTP endpoints for model inference, enabling production deployment patterns like **blue-green** and **canary** releases:

- **Stable URL**: Permanent hostname that persists when underlying services change
- **Traffic Splitting**: Route requests to multiple service endpoints by percentage (canary, A/B testing)
- **Automatic Failover**: Redirects traffic from unhealthy to healthy endpoints
- **Blue-Green Deployment**: Instant cutover between versions by updating gateway targets (zero downtime)

```
                    ┌─────────────────┐
                    │     GATEWAY     │
                    │  (Stable URL)   │
                    └────────┬────────┘
                             │
                             ▼
                    ┌─────────────────┐
                    │    SERVICE      │
                    │  (Model v1)     │
                    │    100%         │
                    └─────────────────┘
```

First, we'll deploy a model as an inference service, then create a gateway to provide a stable endpoint.

### 3a. Deploy Model as Inference Service

Deploy a model version from the registry as a managed SPCS service with HTTP endpoint.

In [ ]:
model_name = "WEEKLY_SALES_FORECAST_SEGMENT_2"
compute_pool = "MLOPS_COMPUTE_POOL"

model = registry.get_model(model_name)
mv_production = model.version("PRODUCTION")

production_version = mv_production.version_name
service_name_production = f"SEGMENT_2_SVC_{production_version}".upper().replace(".", "_")

print(f"🚀 Deploying {model_name} (PRODUCTION) as inference service...\n")

mv_production.create_service(
    service_name=service_name_production,
    service_compute_pool=compute_pool,
    ingress_enabled=True
)

print(f"✅ Service '{service_name_production}' created")
print(f"   Compute Pool: {compute_pool}")
print(f"   Model: {model_name}")
print(f"   Version: {production_version}")

In [ ]:
print("📋 Service Endpoints\n")

services = mv_production.list_services()
print(services)

if len(services) > 0:
    service_info = services.iloc[0]
    inference_endpoint = service_info.get('inference_endpoint', 'N/A')
    internal_endpoint = service_info.get('internal_endpoint', 'N/A')
    
    print(f"\n🔗 Public Endpoint:   {inference_endpoint}")
    print(f"   Internal Endpoint: {internal_endpoint}")

### 3b. Create Gateway

Create a Snowflake Gateway with a stable URL that routes traffic to the inference service.

In [ ]:
gateway_name = "SEGMENT_2_GATEWAY"
db = "ARCA_BEVERAGE_DEMO"
schema = "MODEL_REGISTRY"
service_fqn = f"{db}.{schema}.{service_name_production}"

gateway_sql = f"""
CREATE OR REPLACE GATEWAY {gateway_name}
  FROM SPECIFICATION $$
    spec:
      type: traffic_split
      split_type: custom
      targets:
        - type: endpoint
          value: {service_fqn}!inference
          weight: 100
  $$
"""

print(f"🔧 Creating gateway '{gateway_name}'...\n")
print(f"SQL:\n{gateway_sql}")

session.sql(gateway_sql).collect()
print(f"\n✅ Gateway '{gateway_name}' created")

In [ ]:
import time

print(f"🔍 Gateway Endpoint Details\n")
print(f"Gateway: {gateway_name}")
print(f"Waiting for endpoint provisioning", end="", flush=True)

max_wait_seconds = 300
poll_interval = 10
elapsed = 0

while elapsed < max_wait_seconds:
    gateway_info = session.sql(f"DESC GATEWAY {gateway_name}").collect()[0]
    gateway_endpoint = gateway_info['ingress_url']
    
    if "provisioning" not in gateway_endpoint.lower():
        break
    
    print(".", end="", flush=True)
    time.sleep(poll_interval)
    elapsed += poll_interval

print()

if "provisioning" in gateway_endpoint.lower():
    print(f"⚠️  Endpoint still provisioning after {max_wait_seconds}s")
    print(f"   Status: {gateway_endpoint}")
    print(f"   Run this cell again in a few minutes")
else:
    print(f"\n✅ Endpoint ready! ({elapsed}s)")
    print(f"Stable Endpoint URL: https://{gateway_endpoint}/")
    print(f"\nMethod endpoints:")
    print(f"   Predict: https://{gateway_endpoint}/predict")

### 3c. Call Service from Notebook

When running inside SPCS, use the service's internal endpoint. Get it via `SHOW ENDPOINTS IN SERVICE`.

In [ ]:
import json
import requests

# Get sample data
sample_data = session.table("ARCA_BEVERAGE_DEMO.ML_DATA.TRAINING_DATA").filter(
    "SEGMENT = 'SEGMENT_2'"
).limit(5).to_pandas()

feature_cols = [
    'CUSTOMER_TOTAL_UNITS_4W', 'WEEKS_WITH_PURCHASE', 'VOLUME_QUARTILE',
    'WEEK_OF_YEAR', 'MONTH', 'QUARTER', 'TRANSACTION_COUNT',
    'UNIQUE_PRODUCTS_PURCHASED', 'AVG_UNITS_PER_TRANSACTION'
]

features_df = sample_data[feature_cols]

# Get service details
svc_info = session.sql(f"DESC SERVICE {db}.{schema}.{service_name_production}").collect()[0]
internal_dns = svc_info['dns_name']

print(f"🔗 Internal DNS: {internal_dns}\n")

endpoint_url = f"http://{internal_dns}:5000/predict"

# Submit input data to endpoint
split_obj = json.loads(features_df.to_json(orient="split"))
payload = {"dataframe_split": split_obj}

headers = {"Content-Type": "application/json"}

print(f"🚀 Calling service endpoint: {endpoint_url}\n")

response = requests.post(endpoint_url, headers=headers, json=payload, timeout=30)

if response.status_code == 200:
    result = response.json()
    predictions = result.get('predictions', result)['data']
    
    print("✅ Predictions:\n")
    for i, pred in enumerate(predictions[:5]):
        pred_val = pred[1]['output_feature_0']
        actual = sample_data['WEEKLY_SALES_UNITS'].iloc[i]
        print(f"   Row {i+1}: Predicted={pred_val:.2f}, Actual={actual:.2f}")
else:
    print(f"❌ Request failed: HTTP {response.status_code}")
    print(response.text)

### 3d. Multi-Segment Routing with Partitioned Models

The example above deploys a single segment model. For a unified endpoint that routes to all segments automatically, use the **partitioned wrapper model** from notebook `04f`:

```python
# Deploy the partitioned model (handles all 6 segments internally)
partitioned_model = registry.get_model("WEEKLY_SALES_FORECAST_PARTITIONED")
mv = partitioned_model.version("PRODUCTION")

mv.create_service(
    service_name="WEEKLY_SALES_FORECAST_SVC",
    service_compute_pool="MLOPS_COMPUTE_POOL",
    ingress_enabled=True
)
```

The partitioned model uses `@partitioned_api` to automatically route requests based on the `SEGMENT` column in the input data. A single gateway pointing to this service provides one stable endpoint for all segments.

## 4. Traffic Routing Between Model Versions

With a gateway in place, we can route inference requests to different model versions. This enables:

- **Canary Deployments**: Send a small percentage of traffic to a new version to validate before full rollout
- **A/B Testing**: Compare model performance on live traffic
- **Gradual Rollouts**: Incrementally shift traffic from old to new versions
- **Instant Rollback**: Quickly revert to previous version by adjusting weights

```
Gateway Traffic Routing:
                    ┌─────────────────┐
                    │    GATEWAY      │
                    │  (Stable URL)   │
                    └────────┬────────┘
                             │
              ┌──────────────┴──────────────┐
              │                             │
              ▼                             ▼
        ┌──────────┐                 ┌──────────┐
        │ Model V1 │                 │ Model V2 │
        │   90%    │                 │   10%    │
        └──────────┘                 └──────────┘
         Production                    Canary
```

### 4a. Deploy a Second Model Version as Service

To enable traffic splitting, deploy a second version of the model as its own service.

In [ ]:
mv_candidate = model.version("CANDIDATE")
candidate_version = mv_candidate.version_name
service_name_candidate = f"SEGMENT_2_SVC_{candidate_version}".upper().replace(".", "_")

print(f"🚀 Deploying {model_name} (CANDIDATE) as canary service...\n")

mv_candidate.create_service(
    service_name=service_name_candidate,
    service_compute_pool=compute_pool,
    ingress_enabled=True
)

print(f"✅ Service '{service_name_candidate}' created")
print(f"   Version: {candidate_version}")
print(f"\n📋 Active Services:")
print(f"   PRODUCTION: {service_name_production} ({production_version})")
print(f"   CANDIDATE:  {service_name_candidate} ({candidate_version})")

### 4b. Configure Traffic Split

Use `ALTER GATEWAY` to route traffic between versions. Start with 90% to production and 10% to the canary.

In [ ]:
service_production_fqn = f"{db}.{schema}.{service_name_production}"
service_candidate_fqn = f"{db}.{schema}.{service_name_candidate}"

alter_gateway_sql = f"""
ALTER GATEWAY {gateway_name}
FROM SPECIFICATION $$
spec:
  type: traffic_split
  split_type: custom
  targets:
    - type: endpoint
      value: {service_production_fqn}!inference
      weight: 90
    - type: endpoint
      value: {service_candidate_fqn}!inference
      weight: 10
$$
"""

print(f"🔄 Configuring traffic split for gateway '{gateway_name}'...\n")
print(f"SQL:\n{alter_gateway_sql}")

session.sql(alter_gateway_sql).collect()

print(f"\n✅ Gateway updated:")
print(f"   {service_name_production} (PRODUCTION): 90%")
print(f"   {service_name_candidate} (CANDIDATE):   10%")

### 4c. Gradual Traffic Shift

After validating the canary, gradually increase its traffic share. Once confident, complete the cutover.

In [ ]:
def update_traffic_split(gateway: str, production_weight: int, candidate_weight: int):
    """Update gateway traffic split between production and candidate services."""
    alter_sql = f"""
    ALTER GATEWAY {gateway}
    FROM SPECIFICATION $$
    spec:
      type: traffic_split
      split_type: custom
      targets:
        - type: endpoint
          value: {service_production_fqn}!inference
          weight: {production_weight}
        - type: endpoint
          value: {service_candidate_fqn}!inference
          weight: {candidate_weight}
    $$
    """
    session.sql(alter_sql).collect()
    print(f"✅ Traffic split updated: PRODUCTION={production_weight}%, CANDIDATE={candidate_weight}%")

print("📈 Gradual Traffic Shift Schedule\n")

print("Step 1: Initial canary (90/10)")
update_traffic_split(gateway_name, 90, 10)

print("\nStep 2: Increase canary traffic (60/40)")
update_traffic_split(gateway_name, 60, 40)

print("\nStep 3: Equal split for A/B comparison (50/50)")
update_traffic_split(gateway_name, 50, 50)

print("\nStep 4: Complete cutover to new version (0/100)")
update_traffic_split(gateway_name, 0, 100)

print(f"\n🎉 Migration complete - all traffic now routed to {service_name_candidate}")

### 4d. Shadow Deployment (Request Mirroring)

Unlike traffic splitting where each request goes to ONE target, shadow deployment sends requests to BOTH services:
- **Production** response is returned to the caller
- **Shadow** response is logged for comparison but not returned

This pattern allows validating a new model version on real traffic without any risk to production responses.

```
Shadow Deployment Pattern:
                    ┌─────────────────┐
                    │     CLIENT      │
                    └────────┬────────┘
                             │ request
                             ▼
                    ┌─────────────────┐
                    │  SHADOW PROXY   │
                    └────────┬────────┘
                             │
              ┌──────────────┴──────────────┐
              │ (mirror)                    │ (mirror)
              ▼                             ▼
        ┌──────────┐                 ┌──────────┐
        │ Model V1 │                 │ Model V2 │
        │   100%   │                 │   100%   │
        │ RETURNED │                 │  LOGGED  │
        └──────────┘                 └──────────┘
         Production                    Shadow
```

In [ ]:
import concurrent.futures
from datetime import datetime

def shadow_predict(payload: dict, primary_endpoint: str, shadow_endpoint: str, log_table: str = None):
    """
    Send prediction request to both primary and shadow endpoints.
    Returns primary response; logs shadow response for comparison.
    """
    headers = {"Content-Type": "application/json"}
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        primary_future = executor.submit(
            requests.post, primary_endpoint, headers=headers, json=payload, timeout=30
        )
        shadow_future = executor.submit(
            requests.post, shadow_endpoint, headers=headers, json=payload, timeout=30
        )
        
        primary_response = primary_future.result()
        shadow_response = shadow_future.result()
    
    shadow_log = {
        "timestamp": datetime.now().isoformat(),
        "primary_status": primary_response.status_code,
        "shadow_status": shadow_response.status_code,
        "primary_predictions": primary_response.json() if primary_response.ok else None,
        "shadow_predictions": shadow_response.json() if shadow_response.ok else None,
    }
    
    return primary_response, shadow_log

svc_production_info = session.sql(f"DESC SERVICE {service_production_fqn}").collect()[0]
svc_candidate_info = session.sql(f"DESC SERVICE {service_candidate_fqn}").collect()[0]

endpoint_production = f"http://{svc_production_info['dns_name']}:5000/predict"
endpoint_candidate = f"http://{svc_candidate_info['dns_name']}:5000/predict"

print(f"🔗 Primary (PRODUCTION): {endpoint_production}")
print(f"🔗 Shadow  (CANDIDATE):  {endpoint_candidate}")

In [ ]:
print("🔄 Shadow Deployment Test\n")

split_obj = json.loads(features_df.to_json(orient="split"))
payload = {"dataframe_split": split_obj}

primary_resp, shadow_log = shadow_predict(payload, endpoint_production, endpoint_candidate)

print(f"Primary Status: {shadow_log['primary_status']}")
print(f"Shadow Status:  {shadow_log['shadow_status']}\n")

if primary_resp.ok and shadow_log['shadow_predictions']:
    primary_preds = shadow_log['primary_predictions']['data']
    shadow_preds = shadow_log['shadow_predictions']['data']
    
    print("📊 Prediction Comparison (Production vs Candidate):\n")
    print(f"{'Row':<5} {'Production':>12} {'Candidate':>12} {'Diff':>10}")
    print("-" * 42)
    
    for i, (p, s) in enumerate(zip(primary_preds, shadow_preds)):
        p_val = p[1]['output_feature_0']
        s_val = s[1]['output_feature_0']
        diff = s_val - p_val
        print(f"{i+1:<5} {p_val:>12.2f} {s_val:>12.2f} {diff:>+10.2f}")
    
    print("\n✅ Shadow deployment working - both models received 100% of requests")
    print("   Production response returned to caller; Candidate logged for analysis")
else:
    print(f"❌ Error in shadow deployment test")
    print(f"   Primary: {primary_resp.text if not primary_resp.ok else 'OK'}")

### 4e. Promote Candidate to Production (Gateway Approach)

Unlike model aliases which provide instant version switching for warehouse inference, gateway-based endpoints require explicit service management. The promotion workflow:

1. **Route 100% traffic to CANDIDATE service** (instant, zero downtime)
2. **Update model aliases** to match (for consistency across inference methods)
3. **Retire old service** (optional cleanup)

```
Before Promotion:
┌────────────────────┐     ┌─────────────────────────────────┐
│      GATEWAY       │────▶│ SVC_V_20250210 (PRODUCTION) 0%  │
│  (Stable URL)      │     │ SVC_V_20250211 (CANDIDATE) 100% │
└────────────────────┘     └─────────────────────────────────┘

After Promotion:
┌────────────────────┐     ┌─────────────────────────────────┐
│      GATEWAY       │────▶│ SVC_V_20250211 (PRODUCTION) 100%│
│  (Stable URL)      │     │ SVC_V_20250210 (BACKUP)     0%  │ ← can be retired
└────────────────────┘     └─────────────────────────────────┘
```

In [ ]:
print("🚀 Promoting CANDIDATE to PRODUCTION (Gateway Approach)\n")

promote_gateway_sql = f"""
ALTER GATEWAY {gateway_name}
FROM SPECIFICATION $$
spec:
  type: traffic_split
  split_type: custom
  targets:
    - type: endpoint
      value: {service_candidate_fqn}!inference
      weight: 100
$$
"""

print("Step 1: Route 100% traffic to CANDIDATE service")
session.sql(promote_gateway_sql).collect()
print(f"✅ Gateway now routes to: {service_name_candidate}\n")

print("Step 2: Update model aliases for consistency")
model = registry.get_model(model_name)

old_production_ver = model.version('PRODUCTION').version_name
old_backup_ver = model.version('BACKUP').version_name

model.version(candidate_version).unset_alias('CANDIDATE')
model.version(old_production_ver).unset_alias('PRODUCTION')
model.version(old_backup_ver).unset_alias('BACKUP')

model.version(candidate_version).set_alias('PRODUCTION')
model.version(old_production_ver).set_alias('BACKUP')
model.default = candidate_version

print(f"✅ Aliases updated:")
print(f"   PRODUCTION: {candidate_version} (was CANDIDATE)")
print(f"   BACKUP:     {old_production_ver} (was PRODUCTION)")
print(f"   CANDIDATE:  unassigned (awaiting next version)\n")

print("📋 Current State:")
print(f"   Gateway '{gateway_name}' → {service_name_candidate} (100%)")
print(f"   Model '{model_name}' PRODUCTION alias → {candidate_version}")

### 4f. Service Inventory and Retirement

After promotion, old services remain running and incur compute costs. Review active services and retire those no longer needed.

In [ ]:
print("📋 Active Services Inventory\n")

services_df = session.sql(f"""
    SHOW SERVICES LIKE 'SEGMENT_%_SVC_%' IN COMPUTE POOL {compute_pool}
""").collect()

print(f"{'Service Name':<35} {'Status':<10} {'Instances':<12} {'Pool'}")
print("-" * 85)
for row in services_df:
    if row['is_job'] == 'false':
        instances = f"{row['current_instances']}/{row['target_instances']}"
        print(f"{row['name']:<35} {row['status']:<10} {instances:<12} {row['compute_pool']}")

gateway_desc = session.sql(f"DESC GATEWAY {gateway_name}").collect()[0]
gateway_spec = gateway_desc['spec']

print(f"\n🔗 Gateway '{gateway_name}' configuration:")
print(gateway_spec)

In [ ]:
def retire_service(service_fqn: str, force: bool = False):
    """
    Retire a service that is no longer receiving traffic.
    Checks that service is not actively targeted by gateway before dropping.
    """
    gateway_desc = session.sql(f"DESC GATEWAY {gateway_name}").collect()[0]
    targets_str = str(gateway_desc.get('targets', ''))
    
    service_short = service_fqn.split('.')[-1]
    if service_short.lower() in targets_str.lower() and not force:
        print(f"⚠️  Cannot retire {service_short} - still targeted by gateway")
        print(f"   Use force=True to override (will cause gateway errors)")
        return False
    
    print(f"🗑️  Retiring service: {service_fqn}")
    session.sql(f"DROP SERVICE IF EXISTS {service_fqn}").collect()
    print(f"✅ Service dropped")
    return True

print("📦 Service Retirement Function Defined\n")
print("Usage:")
print(f"   retire_service('{service_production_fqn}')  # Retire old production")
print(f"\n⚠️  Only retire services NOT targeted by the gateway")
print(f"   Old production service: {service_name_production}")

### 4g. Considerations for Partitioned Models (MMT)

When working with Many Model Training (MMT) partitioned models, the gateway architecture differs:

| Aspect | Single Model | Partitioned Model (MMT) |
|--------|--------------|-------------------------|
| **Model Count** | 1 model, multiple versions | N models (one per partition), each with versions |
| **Service Strategy** | One service per version | One service per partition OR single partitioned service |
| **Gateway Routing** | Version-based (V1 vs V2) | Segment-based (route by partition key) |
| **Promotion** | Promote single service | Coordinate promotion across all partitions |

**Partitioned Model Options:**

1. **Single Partitioned Service** (Recommended): Deploy all segment models as one partitioned inference service. Gateway routes all traffic; service routes internally by partition key.

2. **Per-Segment Gateways**: Each segment gets its own gateway + services. Client determines which gateway to call based on segment. More complex but allows independent segment promotion.

3. **Unified Gateway with Segment Routing**: Single gateway with custom routing logic that directs to segment-specific services based on request payload.

For the ARCA demo with 6 segments, Option 1 (partitioned service) is most efficient - it handles partition routing automatically based on the segment column in the input data.

In [ ]:
print("🔄 Promotion Workflow Comparison\n")

print("=" * 70)
print("WAREHOUSE INFERENCE (Aliases)")
print("=" * 70)
print("""
1. Validate CANDIDATE on test data
2. model.version(candidate).set_alias('PRODUCTION')
3. Done - next call to model.version('PRODUCTION').run() uses new version
""")

print("=" * 70)
print("REAL-TIME INFERENCE (Gateway + Services)")  
print("=" * 70)
print("""
1. Deploy CANDIDATE as new service: mv.create_service(...)
2. Add to gateway with low traffic: ALTER GATEWAY ... weight: 10
3. Validate via canary/shadow testing
4. Shift traffic: ALTER GATEWAY ... weight: 100
5. Update aliases for consistency (optional)
6. Retire old service: DROP SERVICE ... (cost savings)
""")

print("=" * 70)
print("Key Difference: Aliases provide instant switching for warehouse")
print("inference. Gateway approach requires service lifecycle management")
print("but enables zero-downtime deployments and instant rollback.")
print("=" * 70)

## 5. Summary

This notebook demonstrated deployment strategies for ML models in Snowflake, from simple alias-based version management to production-grade gateway routing.

### Deployment Options Covered

| Section | Approach | Use Case | Key Benefit |
|---------|----------|----------|-------------|
| **2. Model Aliases** | CANDIDATE/PRODUCTION/BACKUP | Warehouse inference, version control | Instant switching, zero config |
| **3. Gateways** | Stable HTTP endpoints | Real-time inference APIs | Permanent URL survives service changes |
| **4. Traffic Routing** | Weighted splits (90/10 → 0/100) | Canary releases, A/B testing | Gradual rollout, instant rollback |
| **4. Shadow Deployment** | Request mirroring | Validate new models on live traffic | Zero production risk |
| **4. Promotion** | Gateway target swap + alias sync | Production cutover | Zero downtime, consistent state |

### Inference Method Comparison

| Method | Latency | Scaling | Best For |
|--------|---------|---------|----------|
| **Warehouse** (`mv.run()`) | Seconds | Auto (warehouse) | Batch jobs, ad-hoc analysis |
| **Service** (REST API) | Milliseconds | Auto (SPCS) | Real-time apps, external integrations |

### Version Promotion Workflows

| Approach | Steps | Downtime | Rollback Speed |
|----------|-------|----------|----------------|
| **Alias-based** (Warehouse) | `set_alias('PRODUCTION')` | None | Instant |
| **Gateway-based** (Service) | Deploy → Split → Shift → Retire | None | Instant |

### MMT/Partitioned Model Considerations

- Single partitioned service handles segment routing automatically
- Coordinate promotions across all segments for consistency
- Consider per-segment gateways only if independent release cycles needed

## 6. Automated Rollbacks

Production ML systems need automated safeguards to detect model degradation and trigger rollbacks without human intervention. This section covers rollback triggers, thresholds, and implementation patterns.

> **Note**: Model monitoring and observability are covered in detail in **Notebook 06**. This section focuses on the rollback mechanisms that respond to monitoring signals.

### Rollback Triggers

| Trigger Type | Description | Metric | Suggested Threshold |
|--------------|-------------|--------|---------------------|
| **Data Drift** | Input feature distributions shift from training baseline | PSI (Population Stability Index) | PSI > 0.2 (significant drift) |
| **Prediction Drift** | Model output distribution shifts unexpectedly | KL Divergence | KL > 0.1 |
| **Performance Degradation** | Model accuracy drops vs baseline (requires ground truth) | RMSE increase | > 25% above baseline |

**Threshold Guidelines:**
- **Data Drift (PSI)**: < 0.1 = no drift, 0.1-0.2 = moderate drift (monitor), > 0.2 = significant drift (rollback)
- **Prediction Drift (KL)**: Measures divergence between current and baseline prediction distributions
- **Performance (RMSE)**: Compare against baseline established during canary phase; 25% degradation triggers rollback

### Best Practices for Rollback Thresholds

1. **Baseline during canary**: Establish performance baselines with 10% traffic before full rollout
2. **Statistical significance**: Require minimum sample size before triggering (e.g., 1000 predictions)
3. **Cooldown periods**: Prevent rollback thrashing with minimum time between actions (e.g., 15 min)
4. **Severity tiers**: Define WARNING (alert) vs CRITICAL (auto-rollback) thresholds
5. **Ground truth delay**: Performance metrics require actuals; use drift metrics for immediate detection

In [ ]:
ROLLBACK_CONFIG = {
    "data_drift_psi": 0.2,
    "prediction_drift_kl": 0.1,
    "performance_rmse_pct_increase": 0.25,
    "min_samples": 1000,
    "cooldown_minutes": 15,
}

print("⚙️ Rollback Configuration\n")
print(f"   {'Metric':<30} {'Threshold':<15} {'Description'}")
print("   " + "-" * 70)
print(f"   {'data_drift_psi':<30} {ROLLBACK_CONFIG['data_drift_psi']:<15} PSI > 0.2 = significant drift")
print(f"   {'prediction_drift_kl':<30} {ROLLBACK_CONFIG['prediction_drift_kl']:<15} KL divergence from baseline")
print(f"   {'performance_rmse_pct_increase':<30} {ROLLBACK_CONFIG['performance_rmse_pct_increase']:<15.0%} RMSE increase vs baseline")
print(f"   {'min_samples':<30} {ROLLBACK_CONFIG['min_samples']:<15} Minimum predictions before eval")
print(f"   {'cooldown_minutes':<30} {ROLLBACK_CONFIG['cooldown_minutes']:<15} Minutes between rollbacks")

### 6a. Rollback Functions

Two rollback implementations corresponding to the inference methods covered in this notebook:

In [ ]:
def rollback_alias(model_name: str, reason: str = "automated"):
    """
    Rollback for warehouse inference: swap PRODUCTION alias back to BACKUP version.
    
    Use when: Batch inference jobs, warehouse-based predictions (mv.run())
    Speed: Instant - next prediction call uses previous version
    """
    model = registry.get_model(model_name)
    
    current_production = model.version('PRODUCTION').version_name
    backup_version = model.version('BACKUP').version_name
    
    model.version(current_production).unset_alias('PRODUCTION')
    model.version(backup_version).unset_alias('BACKUP')
    
    model.version(backup_version).set_alias('PRODUCTION')
    model.version(current_production).set_alias('BACKUP')
    model.default = backup_version
    
    rollback_log = {
        "timestamp": datetime.now().isoformat(),
        "model": model_name,
        "action": "alias_rollback",
        "from_version": current_production,
        "to_version": backup_version,
        "reason": reason
    }
    
    print(f"🔄 ROLLBACK [{model_name}]: {current_production} → {backup_version}")
    print(f"   Reason: {reason}")
    
    return rollback_log

print("📦 Function defined: rollback_alias(model_name, reason)")
print("   For warehouse inference (mv.run())")

In [ ]:
def rollback_gateway(gateway_name: str, stable_service_fqn: str, reason: str = "automated"):
    """
    Rollback for real-time inference: route 100% traffic to stable service.
    
    Use when: REST API endpoints, real-time predictions via gateway
    Speed: ~1 minute for DNS propagation
    """
    rollback_sql = f"""
    ALTER GATEWAY {gateway_name}
    FROM SPECIFICATION $$
    spec:
      type: traffic_split
      split_type: custom
      targets:
        - type: endpoint
          value: {stable_service_fqn}!inference
          weight: 100
    $$
    """
    
    session.sql(rollback_sql).collect()
    
    rollback_log = {
        "timestamp": datetime.now().isoformat(),
        "gateway": gateway_name,
        "action": "gateway_rollback",
        "routed_to": stable_service_fqn,
        "reason": reason
    }
    
    print(f"🔄 ROLLBACK [{gateway_name}]: 100% → {stable_service_fqn.split('.')[-1]}")
    print(f"   Reason: {reason}")
    
    return rollback_log

print("📦 Function defined: rollback_gateway(gateway_name, stable_service_fqn, reason)")
print("   For real-time inference (REST API)")

### 6b. Automated Rollback Check

This function demonstrates how a monitoring system would evaluate metrics and trigger rollback. In production, this would be called by a scheduled task or event-driven trigger from the observability system (see Notebook 06).

In [ ]:
def check_and_rollback(
    model_name: str,
    gateway_name: str,
    stable_service_fqn: str,
    metrics: dict,
    config: dict = ROLLBACK_CONFIG
) -> dict:
    """
    Evaluate model performance metrics against thresholds and trigger rollback if needed.
    
    Args:
        model_name: Model to rollback (for alias method)
        gateway_name: Gateway to rollback (for service method)
        stable_service_fqn: Service to route traffic to on rollback
        metrics: Current metrics dict with keys: data_drift_psi, prediction_drift_kl,
                 performance_rmse_pct_increase, sample_count
        config: Threshold configuration
    
    Returns:
        Status dict with action taken and reason
    """
    if metrics.get('sample_count', 0) < config['min_samples']:
        return {"action": "skip", "reason": f"Insufficient samples ({metrics.get('sample_count', 0)})"}
    
    triggers = []
    
    if metrics.get('data_drift_psi', 0) > config['data_drift_psi']:
        triggers.append(f"data_drift_psi={metrics['data_drift_psi']:.2f} > {config['data_drift_psi']}")
    
    if metrics.get('prediction_drift_kl', 0) > config['prediction_drift_kl']:
        triggers.append(f"prediction_drift_kl={metrics['prediction_drift_kl']:.3f} > {config['prediction_drift_kl']}")
    
    if metrics.get('performance_rmse_pct_increase', 0) > config['performance_rmse_pct_increase']:
        triggers.append(f"rmse_increase={metrics['performance_rmse_pct_increase']:.0%} > {config['performance_rmse_pct_increase']:.0%}")
    
    if not triggers:
        return {"action": "none", "reason": "All metrics within thresholds"}
    
    reason = "; ".join(triggers)
    
    alias_log = rollback_alias(model_name, reason)
    gateway_log = rollback_gateway(gateway_name, stable_service_fqn, reason)
    
    return {
        "action": "rollback",
        "reason": reason,
        "alias_rollback": alias_log,
        "gateway_rollback": gateway_log
    }

print("📦 Function defined: check_and_rollback(...)")
print("   Evaluates model performance metrics and triggers rollback if thresholds exceeded")

In [ ]:
print("🧪 Simulated Rollback Scenarios\n")

scenarios = [
    {"name": "Healthy", "data_drift_psi": 0.05, "prediction_drift_kl": 0.03, "performance_rmse_pct_increase": 0.05, "sample_count": 5000},
    {"name": "Data Drift", "data_drift_psi": 0.35, "prediction_drift_kl": 0.08, "performance_rmse_pct_increase": 0.10, "sample_count": 5000},
    {"name": "Prediction Drift", "data_drift_psi": 0.10, "prediction_drift_kl": 0.18, "performance_rmse_pct_increase": 0.12, "sample_count": 5000},
    {"name": "Performance Drop", "data_drift_psi": 0.12, "prediction_drift_kl": 0.05, "performance_rmse_pct_increase": 0.40, "sample_count": 5000},
    {"name": "Insufficient Data", "data_drift_psi": 0.50, "prediction_drift_kl": 0.25, "performance_rmse_pct_increase": 0.60, "sample_count": 500},
]

print(f"{'Scenario':<20} {'Data Drift':<12} {'Pred Drift':<12} {'RMSE Δ':<10} {'Action':<10}")
print("-" * 70)

for s in scenarios:
    action = "SKIP" if s['sample_count'] < ROLLBACK_CONFIG['min_samples'] else \
             "ROLLBACK" if (s['data_drift_psi'] > ROLLBACK_CONFIG['data_drift_psi'] or 
                           s['prediction_drift_kl'] > ROLLBACK_CONFIG['prediction_drift_kl'] or
                           s['performance_rmse_pct_increase'] > ROLLBACK_CONFIG['performance_rmse_pct_increase']) else "OK"
    
    print(f"{s['name']:<20} {s['data_drift_psi']:<12.2f} {s['prediction_drift_kl']:<12.3f} {s['performance_rmse_pct_increase']:<10.0%} {action:<10}")

print(f"\nThresholds: PSI > {ROLLBACK_CONFIG['data_drift_psi']}, KL > {ROLLBACK_CONFIG['prediction_drift_kl']}, RMSE Δ > {ROLLBACK_CONFIG['performance_rmse_pct_increase']:.0%}")
print("\n⚠️  Note: Actual rollback calls commented out for demonstration")
print("   In production, check_and_rollback() would execute the rollback")

---

## Notebook 05: Deployment Options - Complete

**Sections covered:**
1. Model Registry Setup - Verified 6 segment models with multiple versions
2. Model Aliases - CANDIDATE → PRODUCTION → BACKUP lifecycle
3. Gateways - Stable endpoints for real-time inference
4. Traffic Routing - Canary, shadow deployment, promotion
5. Summary - Deployment patterns comparison
6. Automated Rollbacks - Triggers, thresholds, rollback functions

**Functions defined:**
- `rollback_alias()` - Instant version swap for warehouse inference
- `rollback_gateway()` - Traffic rerouting for real-time inference
- `check_and_rollback()` - Automated threshold evaluation

**Next steps:**
- → **05b**: Batch inference pipelines
- → **06**: ML Observability and monitoring (rollback triggers)